# Finetune and deploy Cohere Classification Models from AWS Marketplace

This sample notebook shows you how to finetune and deploy classification models using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [cohere-classification-finetuning-multilingual](https://aws.amazon.com/marketplace/pp/prodview-hukrkor45t7bw). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Finetune Classification Models](#2.-Finetune-Classification-Models)
   1. [Upload training data](#A.-Upload-training-data)
   2. [Finetune models on uploaded data](#B.-Finetune-models-on-uploaded-data)
3. [Create an endpoint for inference with multiple models](#3.-Create-an-endpoint-for-inference-with-multiple-models)
   1. [Create an endpoint]()
   2. [Perform real-time inference]()
4. [Clean-up](#4.-Clean-up)
    1. [Delete the endpoint](#A.-Delete-the-endpoint)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the algorithm

To subscribe to the model package:
1. Open the algorithm listing page [cohere-classification-finetuning-multilingual](https://aws.amazon.com/marketplace/pp/prodview-hukrkor45t7bw)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the algorithm ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
!pip install --upgrade cohere-sagemaker
# if you upgrade the package, you need to restart the kernel

from cohere_sagemaker import Client
import boto3
import sagemaker as sage
from sagemaker.s3 import S3Uploader

In [ ]:
region = boto3.Session().region_name

cohere_package = "classification-finetuning-mult-8ebd7456a612378ba0218d26fa9d25da"

algorithm_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:algorithm/{cohere_package}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:algorithm/{cohere_package}",
    "us-west-1": f"arn:aws:sagemaker:us-west-1:382657785993:algorithm/{cohere_package}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:algorithm/{cohere_package}",
    "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:algorithm/{cohere_package}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:algorithm/{cohere_package}",
    "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:algorithm/{cohere_package}",
    "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:algorithm/{cohere_package}",
    "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:algorithm/{cohere_package}",
    "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:algorithm/{cohere_package}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:algorithm/{cohere_package}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:algorithm/{cohere_package}",
    "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:algorithm/{cohere_package}",
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:algorithm/{cohere_package}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:algorithm/{cohere_package}",
    "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:algorithm/{cohere_package}",
}
if region not in algorithm_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

arn = algorithm_map[region]

## 2. Finetune classification models

### A. Upload training data

Choose a directory on S3 to store the training data:

In [ ]:
s3_data_dir = "s3://..."  # Do not add a trailing slash otherwise the upload will not work

Upload sample training data to S3:

In [ ]:
sess = sage.Session()
train_dataset1 = S3Uploader.upload("../examples/sample_sentiment_classification_data.jsonl", s3_data_dir, sagemaker_session=sess)
train_dataset2 = S3Uploader.upload("../examples/sample_multilabel_classification_data.jsonl", s3_data_dir, sagemaker_session=sess)

### B. Finetune models on uploaded data

Specify a directory on S3 where finetuned models should be stored:

In [ ]:
s3_models_dir = "s3://..."  # Do not add a trailing slash otherwise it will not work

Create Cohere client:

In [ ]:
co = Client(region_name=region)

Create fine-tuning jobs for both uploaded datasets:

In [ ]:
co.create_finetune(arn=arn, name="model1", train_data=train_dataset1, s3_models_dir=s3_models_dir)

In [ ]:
co.create_finetune(arn=arn, name="model2", train_data=train_dataset2, s3_models_dir=s3_models_dir)

## 3. Create an endpoint for inference with multiple models

### A. Create an endpoint

The Cohere SageMaker SDK provides a built-in method for creating an endpoint for inference. This will automatically deploy all models you finetuned earlier.

> **Note**: This is equivalent to creating and deploying a `ModelPackage` in SageMaker's SDK.

In [ ]:
co.create_endpoint(arn=arn, endpoint_name="cohere-classify", s3_models_dir=s3_models_dir, recreate=True)

# # If the endpoint is already created, you just need to connect to it
# co.connect_to_endpoint(endpoint_name="cohere-classify")

### B. Perform real-time inference

Now, you can access all models deployed on the endpoint for inference:

In [ ]:
print(co.classify(["it works!", "cela fonctionne!"], "model1"))

In [ ]:
print(co.classify(["Vamos a la pizzeria"], "model2"))

## 4. Clean-up

### A. Delete the endpoint

After you've successfully performed inference, you can delete the deployed endpoint to avoid being charged continuously. This can also be done via the Cohere SageMaker SDK:

In [ ]:
co.delete_endpoint()
co.close()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

